In [338]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

In [384]:
mydata = pd.read_csv("card transactions.csv")
mydata = mydata.iloc[:, :10]
mydata.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,1/1/10,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,1/1/10,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,1/1/10,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [385]:
mydata.shape

(96753, 10)

## 1. Indentify exclusion and bad records
1. Single large transaction outlier
2. All but the "P" type transactions

In [386]:
# exclude a record that has an extremely large transaction amount
mydata = mydata.sort_values("Amount", ascending=False).iloc[1:, :]
mydata.shape

(96752, 10)

In [387]:
# only remain records whose Transtype is "P"
mydata = mydata.loc[mydata["Transtype"] == "P", :].sort_values("Recnum")
mydata.shape

(96397, 10)

## 2. Fill in missing values (Zip, Merchnum, State)

In [388]:
mydata.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,1/1/10,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,1/1/10,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,1/1/10,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [389]:
mydata.isnull().sum()

Recnum                  0
Cardnum                 0
Date                    0
Merchnum             3198
Merch description       0
Merch state          1020
Merch zip            4300
Transtype               0
Amount                  0
Fraud                   0
dtype: int64

In [392]:
# replace zero values with NaN
mydata["Merchnum"] = np.where(mydata["Merchnum"] == "0", np.NaN, mydata["Merchnum"])

In [393]:
mydata.isnull().sum()

Recnum                  0
Cardnum                 0
Date                    0
Merchnum             3251
Merch description       0
Merch state          1020
Merch zip            4300
Transtype               0
Amount                  0
Fraud                   0
dtype: int64

In [394]:
# change ["Cardnum", "Merchnum", "Merch state", "Merch zip"] to string type
col = ["Cardnum", "Merchnum", "Merch state", "Merch zip"]
for c in col:
    mydata[c] = mydata[c].astype(str)

In [395]:
def most_frequent(x):
    """
    extract the index of the most frequent element
    """
    try:
        return x.value_counts().idxmax()
    except:
        return "nan"

### Replace Merchnum missing values

In [396]:
# Merchnum
# replace nan in merchnum with the most common value grouped by merch description
init = len(mydata[mydata["Merchnum"] == "nan"])
mydata['Most_common_merch_per_descrip'] = mydata['Merch description'].map(mydata.groupby('Merch description')['Merchnum'].agg(lambda x: most_frequent(x)))
mydata["Merchnum"] = np.where(mydata["Merchnum"] == "nan", mydata["Most_common_merch_per_descrip"], mydata["Merchnum"])
nan = len(mydata[mydata["Merchnum"] == "nan"])
print(f"Initial: {init} null values, after replacing by linking merch description, Merchnum still have {nan} null values.")
# replace nan in merchnum grouped by cardnum
init = len(mydata[mydata["Merchnum"] == "nan"])
mydata['Most_common_merch_per_card'] = mydata['Cardnum'].map(mydata.groupby('Cardnum')['Merchnum'].agg(lambda x: most_frequent(x)))
mydata["Merchnum"] = np.where(mydata["Merchnum"] == "nan", mydata["Most_common_merch_per_card"], mydata["Merchnum"])
nan = len(mydata[mydata["Merchnum"] == "nan"])
print(f"Initial: {init} null values, after replacing by linking Cardnum, Merchnum still have {nan} null values.")
# the rest of the nan in merchnum, replace with record number
mydata["Merchnum"] = np.where(mydata["Merchnum"] == "nan", mydata["Recnum"], mydata["Merchnum"])
nan = len(mydata[mydata["Merchnum"] == "nan"])
print(f"After replacing with record number, Merchnum has {nan} null values.")

Initial: 3251 null values, after replacing by linking merch description, Merchnum still have 2834 null values.
Initial: 2834 null values, after replacing by linking Cardnum, Merchnum still have 745 null values.
After replacing with record number, Merchnum has 0 null values.


### Replace Merch state & Merch zip missing values

In [397]:
# replace nan in state & zip with the most common value grouped by merchnum
init_state = len(mydata[mydata["Merch state"] == "nan"])
init_zip = len(mydata[mydata["Merch zip"] == "nan"])
mydata['Most_common_state'] = mydata['Merchnum'].map(mydata.groupby('Merchnum')['Merch state'].agg(lambda x: most_frequent(x)))
mydata['Most_common_zip'] = mydata['Merchnum'].map(mydata.groupby('Merchnum')['Merch zip'].agg(lambda x: most_frequent(x)))
mydata["Merch state"] = np.where(mydata["Merch state"] == "nan", mydata["Most_common_state"], mydata["Merch state"])
mydata["Merch zip"] = np.where(mydata["Merch zip"] == "nan", mydata["Most_common_zip"], mydata["Merch zip"])
nan_state = len(mydata[mydata["Merch state"] == "nan"])
print(f"Initial: {init_state} null values, after replacing by linking Merchnum, Merch state still have {nan_state} null values.")
nan_zip = len(mydata[mydata["Merch zip"] == "nan"])
print(f"Initial: {init_zip} null values, after replacing by linking Merchnum, Merch zip still have {nan_zip} null values.")
# the rest of the nan, replace with record number
mydata["Merch state"] = np.where(mydata["Merch state"] == "nan", mydata["Recnum"], mydata["Merch state"])
mydata["Merch zip"] = np.where(mydata["Merch zip"] == "nan", mydata["Recnum"], mydata["Merch zip"])
nan_state = len(mydata[mydata["Merch state"] == "nan"])
print(f"After replacing with record number, Merch state has {nan} null values.")
nan_zip = len(mydata[mydata["Merch zip"] == "nan"])
print(f"After replacing with record number, Merch zip has {nan} null values.")

Initial: 1020 null values, after replacing by linking Merchnum, Merch state still have 405 null values.
Initial: 4300 null values, after replacing by linking Merchnum, Merch zip still have 2685 null values.
After replacing with record number, Merch state has 0 null values.
After replacing with record number, Merch zip has 0 null values.


In [398]:
mydata["Date"] = pd.to_datetime(mydata["Date"], format="%m/%d/%y")
mydata.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,Most_common_merch_per_descrip,Most_common_merch_per_card,Most_common_state,Most_common_zip
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,5509006296254,930090121224,TN,38118.0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,61003026333,6005030600003,MA,1803.0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,4503082993600,7593860068556,MD,20706.0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,5509006296254,930090121224,TN,38118.0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,5509006296254,930090121224,TN,38118.0


In [399]:
# change ["Cardnum", "Merchnum", "Merch state", "Merch zip"] to string type
col = ["Cardnum", "Merchnum", "Merch state", "Merch zip"]
for c in col:
    mydata[c] = mydata[c].astype(str)
# create other three entity groups
mydata["card_merch"] = mydata["Cardnum"] + "_" + mydata["Merchnum"]
mydata["card_zip"] = mydata["Cardnum"] + "_" + mydata["Merch zip"]
mydata["card_state"] = mydata["Cardnum"] + "_" + mydata["Merch state"]

In [400]:
mydata.to_csv("transaction_cleaned.csv", index=False)

## 3. Create Variables
### 1. 240 Amount Variables
### 2. 30 Frequency Variables
### 3. 5 Days-since Variables
### 4. 48 Velocity change Variables
### 5. 2 Benford’s law variables (merchnum & cardnum)
### 6. 1 day-of-week risk table

In [401]:
# Create All Variables
start_time = pd.datetime.now()

# Entity
entities = ["Cardnum", "Merchnum", "card_merch", "card_zip", "card_state"]
# Time Window
days = [0, 1, 3, 7, 14, 30]
# Save column names in lists
amount = []
daysince = []
frequency = []
daily = []
velocity = []

# create copy of mydata
df = mydata.copy()
final_df = mydata.copy()
# append new variables to final_df dataframe, merge by "Recnum"
final_df = final_df[["Recnum", "Amount"]]

for en in entities:
    # select only the necessary columns for Daysince and Frequency Variables
    df1 = df[["Recnum", "Date", en]]
    df2 = df1
    
    # select only the necessary columns for Amount Variables
    df1_a = df[["Recnum", "Date", "Amount", en]]
    df2_a = df1_a
    
    # Daysince Variables
    # self-join
    tmp = df1.merge(df2, on=[en])
    # filter the records that are in the past for each record
    window = (tmp["Recnum_x"] > tmp["Recnum_y"])
    # for each record, select the most recent record with the same entity value
    tmp = tmp.loc[window, :].sort_values([en, "Recnum_x", "Date_y"])
    tmp = tmp.groupby([en, "Recnum_x"])["Date_x", "Date_y"].last().reset_index()
    
    newcol = en+"_daysince"
    daysince.append(newcol)
    # calculate the difference of two dates
    tmp[newcol] = (tmp["Date_x"] - tmp["Date_y"]) / np.timedelta64(1, "D")
    # append to final_df, and replace all the value that first appears with 365
    final_df = final_df.merge(tmp, left_on="Recnum", right_on="Recnum_x", how="left")
    final_df = final_df.drop(columns=[en, "Recnum_x", "Date_x", "Date_y"]).fillna(365.0)
    print("done"+newcol)
    
    for day in days:
        # Frequency Variables
        #df1 = df[["Recnum", "Date", en]]
        #df2 = df1
        tmp = df1.merge(df2, on=[en])
        # filter the records that are in the past and also over the past n days
        window = (tmp["Recnum_x"] > tmp["Recnum_y"]) & (tmp["Date_x"] - pd.Timedelta(days=day) <= tmp["Date_y"])
        # count the nunber of records partitioned by entity values
        tmp = tmp.loc[window, :].groupby([en, "Recnum_x"])["Recnum_y"].count().reset_index()
        
        newcol = en+"_fre_"+str(day)
        frequency.append(newcol)
        tmp = tmp.rename(columns={"Recnum_y": newcol})
        final_df = final_df.merge(tmp, left_on="Recnum", right_on="Recnum_x", how="left").drop(columns=[en, "Recnum_x"])
        final_df[newcol] = final_df[newcol].fillna(0)
        print("done"+newcol)
        
        # Amount Variables
        tmp = df1_a.merge(df2_a, on=[en])
        # filter the records that are in the past and also over the past n days
        window = (tmp["Recnum_x"] > tmp["Recnum_y"]) & (tmp["Date_x"] - pd.Timedelta(days=day) <= tmp["Date_y"])
        # calculate the (mean, max, median, sum) of Amount partitioned by entity values
        df_mean = tmp.loc[window, :].groupby([en, "Recnum_x"])["Amount_y"].mean().reset_index()
        df_max = tmp.loc[window, :].groupby([en, "Recnum_x"])["Amount_y"].max().reset_index()
        df_median = tmp.loc[window, :].groupby([en, "Recnum_x"])["Amount_y"].median().reset_index()
        df_total = tmp.loc[window, :].groupby([en, "Recnum_x"])["Amount_y"].sum().reset_index()
        
        amount_dict = {"_mean_": df_mean, "_max_": df_max, "_median_": df_median, "_total_": df_total}
        for i in amount_dict:
            newcol = en+i+str(day)
            amount.append(newcol)
            amount_dict[i] = amount_dict[i].rename(columns={"Amount_y": newcol})
            final_df = final_df.merge(amount_dict[i], left_on="Recnum", right_on="Recnum_x", how="left").drop(columns=[en, "Recnum_x"])
            final_df[newcol] = final_df[newcol].fillna(0)
            
            # Actual Amount/(mean, max, median, total) variables
            # filter the records that are not zero in order to avoid zero to appear in the denominator
            actual = en+"_actual/"+i+str(day)
            amount.append(actual)
            tmp = final_df.loc[final_df[newcol] != 0, ["Recnum", "Amount", newcol]]
            tmp[actual] = tmp["Amount"] / tmp[newcol]
            tmp = tmp.drop(columns=["Amount", newcol])
            final_df = final_df.merge(tmp, on="Recnum", how="left")
            # replace NaN with zero
            final_df[actual] = final_df[actual].fillna(0)
            
            print("done"+newcol)

# Velocity change Variables
for en1 in entities[:2]:
    #for en2 in entities[:2]:
    for day1 in days[:2]:
        for day2 in days[3:]:
            # number of transaction / Average daily number of transactions
            newcol = "num/num" + str(day1) + en1 + str(day2)
            velocity.append(newcol)
            final_df[newcol] = final_df[en1+"_fre_"+str(day1)] / (final_df[en1+"_fre_"+str(day2)] / day2)
            # number of transaction / Average daily amount of transactions
            newcol = "num/amount" + str(day1) + en1 + str(day2)
            velocity.append(newcol)
            final_df[newcol] = final_df[en1+"_fre_"+str(day1)] / (final_df[en1+"_total_"+str(day2)] / day2)
            # amount of transaction / Average daily number of transactions
            newcol = "amount/num" + str(day1) + en1 + str(day2)
            velocity.append(newcol)
            final_df[newcol] = final_df[en1+"_total_"+str(day1)] / (final_df[en1+"_fre_"+str(day2)] / day2)
            # amount of transaction / Average daily amount of transactions
            newcol = "amount/amount" + str(day1) + en1 + str(day2)
            velocity.append(newcol)
            final_df[newcol] = final_df[en1+"_total_"+str(day1)] / (final_df[en1+"_total_"+str(day2)] / day2)
                
            print(newcol)
end_time = pd.datetime.now()
print(start_time)
print(end_time)
print("The time taken to create all variables is:")
print(end_time - start_time)

/Users/wanwanjong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  
/Users/wanwanjong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


doneCardnum_daysince
doneCardnum_fre_0
doneCardnum_mean_0
doneCardnum_max_0
doneCardnum_median_0
doneCardnum_total_0
doneCardnum_fre_1
doneCardnum_mean_1
doneCardnum_max_1
doneCardnum_median_1
doneCardnum_total_1
doneCardnum_fre_3
doneCardnum_mean_3
doneCardnum_max_3
doneCardnum_median_3
doneCardnum_total_3
doneCardnum_fre_7
doneCardnum_mean_7
doneCardnum_max_7
doneCardnum_median_7
doneCardnum_total_7
doneCardnum_fre_14
doneCardnum_mean_14
doneCardnum_max_14
doneCardnum_median_14
doneCardnum_total_14
doneCardnum_fre_30
doneCardnum_mean_30
doneCardnum_max_30
doneCardnum_median_30
doneCardnum_total_30
doneMerchnum_daysince
doneMerchnum_fre_0
doneMerchnum_mean_0
doneMerchnum_max_0
doneMerchnum_median_0
doneMerchnum_total_0
doneMerchnum_fre_1
doneMerchnum_mean_1
doneMerchnum_max_1
doneMerchnum_median_1
doneMerchnum_total_1
doneMerchnum_fre_3
doneMerchnum_mean_3
doneMerchnum_max_3
doneMerchnum_median_3
doneMerchnum_total_3
doneMerchnum_fre_7
doneMerchnum_mean_7
doneMerchnum_max_7
doneMerchn

/Users/wanwanjong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:119: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.


In [402]:
final_df.head()

,Recnum,Amount,Cardnum_daysince,Cardnum_fre_0,Cardnum_mean_0,Cardnum_actual/_mean_0,Cardnum_max_0,Cardnum_actual/_max_0,Cardnum_median_0,Cardnum_actual/_median_0,...,amount/num1Merchnum7,amount/amount1Merchnum7,num/num1Merchnum14,num/amount1Merchnum14,amount/num1Merchnum14,amount/amount1Merchnum14,num/num1Merchnum30,num/amount1Merchnum30,amount/num1Merchnum30,amount/amount1Merchnum30
0,1,3.62,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,31.42,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,178.49,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,3.62,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,25.34,7.0,14.0,3.867403,50.68,14.0,30.0,8.287293,108.6,30.0
4,5,3.62,0.0,1.0,3.62,1.0,3.62,1.0,3.62,1.0,...,25.34,7.0,14.0,3.867403,50.68,14.0,30.0,8.287293,108.6,30.0


In [297]:
# final_df.to_csv("variables_without_benford.csv", index=False)

In [403]:
# week risk table
# Only use data from Jan to Oct to calculate the fraud percentage
Jan_Oct = mydata[mydata['Date'] < '2010-11-01']
Jan_Oct['weekday'] = Jan_Oct['Date'].apply(lambda x: x.weekday())

# calculate average of each group using smoothing function
c,nmid = 4,20
y_avg = Jan_Oct['Fraud'].mean()
y_weekday_avg = Jan_Oct.groupby('weekday')['Fraud'].mean()
num_instances_weekday = Jan_Oct.groupby('weekday').size()
y_weekday_smooth = y_avg+(y_weekday_avg-y_avg)/(1+np.exp(-(num_instances_weekday-nmid)/c))
Jan_Oct['weekday_risk'] = Jan_Oct['weekday'].map(y_weekday_smooth)

week = pd.DataFrame(Jan_Oct.groupby('weekday')['weekday_risk'].mean())
week.reset_index(inplace=True)
week['week_day']=week['weekday'].map({0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'})
week

/Users/wanwanjong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/wanwanjong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,weekday,weekday_risk,week_day
0,0,0.008711,Monday
1,1,0.007127,Tuesday
2,2,0.009788,Wednesday
3,3,0.018626,Thursday
4,4,0.025994,Friday
5,5,0.010095,Saturday
6,6,0.009630,Sunday


In [404]:
# apply the value from the risk table to the whole dataset (Jan to Dec)
tmp = mydata[["Recnum", "Date"]]
tmp['weekday'] = tmp['Date'].apply(lambda x: x.weekday())
dayrisk = {0: 0.008711, 1: 0.007127, 2: 0.009788, 3: 0.018626, 4:0.025994 , 5:0.010095, 6:0.009630}
tmp['risk_week']=tmp["weekday"].apply(lambda day: dayrisk[day])
tmp.head()

/Users/wanwanjong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/wanwanjong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Recnum,Date,weekday,risk_week
0,1,2010-01-01,4,0.025994
1,2,2010-01-01,4,0.025994
2,3,2010-01-01,4,0.025994
3,4,2010-01-01,4,0.025994
4,5,2010-01-01,4,0.025994


In [405]:
tmp = tmp.drop(columns=["Date", "weekday"])
final_df = final_df.merge(tmp, on="Recnum")
final_df.head()

,Recnum,Amount,Cardnum_daysince,Cardnum_fre_0,Cardnum_mean_0,Cardnum_actual/_mean_0,Cardnum_max_0,Cardnum_actual/_max_0,Cardnum_median_0,Cardnum_actual/_median_0,...,amount/amount1Merchnum7,num/num1Merchnum14,num/amount1Merchnum14,amount/num1Merchnum14,amount/amount1Merchnum14,num/num1Merchnum30,num/amount1Merchnum30,amount/num1Merchnum30,amount/amount1Merchnum30,risk_week
0,1,3.62,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025994
1,2,31.42,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025994
2,3,178.49,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025994
3,4,3.62,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,7.0,14.0,3.867403,50.68,14.0,30.0,8.287293,108.6,30.0,0.025994
4,5,3.62,0.0,1.0,3.62,1.0,3.62,1.0,3.62,1.0,...,7.0,14.0,3.867403,50.68,14.0,30.0,8.287293,108.6,30.0,0.025994


In [406]:
# fill nan values with zero for velocity change variables
for col in velocity:
    final_df[col] = final_df[col].fillna(0)
final_df.head()

,Recnum,Amount,Cardnum_daysince,Cardnum_fre_0,Cardnum_mean_0,Cardnum_actual/_mean_0,Cardnum_max_0,Cardnum_actual/_max_0,Cardnum_median_0,Cardnum_actual/_median_0,...,amount/amount1Merchnum7,num/num1Merchnum14,num/amount1Merchnum14,amount/num1Merchnum14,amount/amount1Merchnum14,num/num1Merchnum30,num/amount1Merchnum30,amount/num1Merchnum30,amount/amount1Merchnum30,risk_week
0,1,3.62,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.025994
1,2,31.42,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.025994
2,3,178.49,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.025994
3,4,3.62,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,7.0,14.0,3.867403,50.68,14.0,30.0,8.287293,108.6,30.0,0.025994
4,5,3.62,0.0,1.0,3.62,1.0,3.62,1.0,3.62,1.0,...,7.0,14.0,3.867403,50.68,14.0,30.0,8.287293,108.6,30.0,0.025994


In [407]:
final_df.to_csv("variables_without_benford.csv", index=False)

In [408]:
final_df.shape

(96397, 326)

## Benfords Law Variables
Not Done Yet!!!

In [412]:
final = pd.read_csv("variables_without_benford.csv")
ben = pd.read_csv("variables_with_benford.csv")
ben.head()

,Recnum,Amount,Cardnum_daysince,Cardnum_fre_0,Cardnum_mean_0,Cardnum_actual/_mean_0,Cardnum_max_0,Cardnum_actual/_max_0,Cardnum_median_0,Cardnum_actual/_median_0,...,num/amount1Merchnum14,amount/num1Merchnum14,amount/amount1Merchnum14,num/num1Merchnum30,num/amount1Merchnum30,amount/num1Merchnum30,amount/amount1Merchnum30,risk_week,Benford_Card,Benford_Merch
0,1,3.62,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.025994,1.000000,1.000000
1,2,31.42,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.025994,1.000894,1.000894
2,3,178.49,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.025994,1.000894,1.000894
3,4,3.62,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,3.867403,50.68,14.0,30.0,8.287293,108.6,30.0,0.025994,1.000000,1.000000
4,5,3.62,0.0,1.0,3.62,1.0,3.62,1.0,3.62,1.0,...,3.867403,50.68,14.0,30.0,8.287293,108.6,30.0,0.025994,1.000000,1.000000


In [414]:
final = final.merge(ben[["Recnum", "Benford_Card", "Benford_Merch"]], on="Recnum", how="left")
final.head()

,Recnum,Amount,Cardnum_daysince,Cardnum_fre_0,Cardnum_mean_0,Cardnum_actual/_mean_0,Cardnum_max_0,Cardnum_actual/_max_0,Cardnum_median_0,Cardnum_actual/_median_0,...,num/amount1Merchnum14,amount/num1Merchnum14,amount/amount1Merchnum14,num/num1Merchnum30,num/amount1Merchnum30,amount/num1Merchnum30,amount/amount1Merchnum30,risk_week,Benford_Card,Benford_Merch
0,1,3.62,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.025994,1.000000,1.000000
1,2,31.42,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.025994,1.000894,1.000894
2,3,178.49,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.025994,1.000894,1.000894
3,4,3.62,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,3.867403,50.68,14.0,30.0,8.287293,108.6,30.0,0.025994,1.000000,1.000000
4,5,3.62,0.0,1.0,3.62,1.0,3.62,1.0,3.62,1.0,...,3.867403,50.68,14.0,30.0,8.287293,108.6,30.0,0.025994,1.000000,1.000000


In [415]:
final.to_csv("variables_with_benford.csv", index=False)

## Not Done Yet

In [287]:
# Benfords Law variables
card_ben = pd.read_excel("card_benford.xlsx")
merch_ben = pd.read_excel("merch_benford.xlsx")
card_ben = card_ben.rename(columns={"U*": "Cardnum_U*"})
merch_ben = merch_ben.rename(columns={"U*": "Merchnum_U*"})
card_ben["Cardnum"] = card_ben["Cardnum"].astype(str)
merch_ben["Merchnum"] = merch_ben["Merchnum"].astype(str)

In [293]:
card_ben.shape

(1642, 9)

In [294]:
merch_ben.shape

(13084, 9)

In [290]:
all_df = final_df.copy()
all_df = pd.merge(all_df, mydata[["Recnum", "Cardnum", "Merchnum"]], on="Recnum", how="left")
all_df = pd.merge(all_df, card_ben[["Cardnum", "Cardnum_U*"]], on="Cardnum", how="left")
all_df = pd.merge(all_df, merch_ben[["Merchnum", "Merchnum_U*"]], on="Merchnum", how="left")
all_df.head()

,Recnum,Amount,Cardnum_daysince,Cardnum_fre_0,Cardnum_mean_0,Cardnum_actual/_mean_0,Cardnum_max_0,Cardnum_actual/_max_0,Cardnum_median_0,Cardnum_actual/_median_0,...,amount/numMerchnum1Merchnum14,amount/amountMerchnum1Merchnum14,num/numMerchnum1Merchnum30,num/amountMerchnum1Merchnum30,amount/numMerchnum1Merchnum30,amount/amountMerchnum1Merchnum30,Cardnum,Merchnum,Cardnum_U*,Merchnum_U*
0,1,3.62,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,5142190439,5509006296254,2.178008,NaN
1,2,31.42,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,5142183973,61003026333,1.604857,1.000894
2,3,178.49,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,5142131721,4503082993600,2.368143,1.025818
3,4,3.62,365.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,50.68,14.0,30.0,8.287293,108.6,30.0,5142148452,5509006296254,1.044105,NaN
4,5,3.62,0.0,1.0,3.62,1.0,3.62,1.0,3.62,1.0,...,50.68,14.0,30.0,8.287293,108.6,30.0,5142190439,5509006296254,2.178008,NaN


In [291]:
all_df["Cardnum_U*"].isnull().sum()

72

In [292]:
all_df["Merchnum_U*"].isnull().sum()

12756

In [303]:
# replace NaN values!!!
velocity
# Benford's law exclude FEDEX values

['num/numCardnum0Cardnum7',
 'num/amountCardnum0Cardnum7',
 'amount/numCardnum0Cardnum7',
 'amount/amountCardnum0Cardnum7',
 'num/numCardnum0Cardnum14',
 'num/amountCardnum0Cardnum14',
 'amount/numCardnum0Cardnum14',
 'amount/amountCardnum0Cardnum14',
 'num/numCardnum0Cardnum30',
 'num/amountCardnum0Cardnum30',
 'amount/numCardnum0Cardnum30',
 'amount/amountCardnum0Cardnum30',
 'num/numCardnum1Cardnum7',
 'num/amountCardnum1Cardnum7',
 'amount/numCardnum1Cardnum7',
 'amount/amountCardnum1Cardnum7',
 'num/numCardnum1Cardnum14',
 'num/amountCardnum1Cardnum14',
 'amount/numCardnum1Cardnum14',
 'amount/amountCardnum1Cardnum14',
 'num/numCardnum1Cardnum30',
 'num/amountCardnum1Cardnum30',
 'amount/numCardnum1Cardnum30',
 'amount/amountCardnum1Cardnum30',
 'num/numCardnum0Merchnum7',
 'num/amountCardnum0Merchnum7',
 'amount/numCardnum0Merchnum7',
 'amount/amountCardnum0Merchnum7',
 'num/numCardnum0Merchnum14',
 'num/amountCardnum0Merchnum14',
 'amount/numCardnum0Merchnum14',
 'amount/amountC